In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl

np.random.seed(1)

In [ ]:
NUMBER_OF_CLASSES = 10
EPOCHS = 500
BATCH_SIZE = 256
VALIDATION_SPLIT = .2

In [ ]:
# Load MNIST dataset
(X_train, Y_train), (X_test, Y_test) = keras.datasets.mnist.load_data()

In [ ]:
idx = [3, 6, 7, 1, 9, 100]  # Example indices
fig, axs = plt.subplots(2, 3)
axs[0, 0].imshow(X_train[idx[0]])
axs[0, 1].imshow(X_train[idx[1]])
axs[0, 2].imshow(X_train[idx[2]])
axs[1, 0].imshow(X_train[idx[3]])
axs[1, 1].imshow(X_train[idx[4]])
axs[1, 2].imshow(X_train[idx[5]])

In [ ]:
print("X_train shape:", X_train.shape)
print("Y_train shape:", Y_train.shape)
print("Y_train classes: %i classes" % np.unique(Y_train).size)
print("X_test shape:", X_test.shape)
print("Y_test shape:", Y_test.shape)

print("Number of training examples:", X_train.shape[0])
print("Number of testing examples: ", X_test.shape[0])
print("Number of unique classes: %i classes" % np.unique(Y_train).size)
print("Each image is of size: %i x %i" % X_train.shape[1:3])

In [ ]:
X_train_preprocessed = X_train.reshape(X_train.shape[0], -1)
X_test_preprocessed = X_test.reshape(X_test.shape[0], -1)
print("Flatten shape of X_train: ", X_train_preprocessed.shape)
print("Flatten shape of X_test: ", X_test_preprocessed.shape)

In [ ]:
Y_train_preprocessed = keras.utils.to_categorical(Y_train, NUMBER_OF_CLASSES)
Y_test_preprocessed = keras.utils.to_categorical(Y_test, NUMBER_OF_CLASSES)
print("Current shape of Y_train_preprocessed:", Y_train_preprocessed.shape)
print("Current shape of Y_test_preprocessed:", Y_test_preprocessed.shape)

In [ ]:
X_train_preprocessed = X_train_preprocessed / 255
X_test_preprocessed = X_test_preprocessed / 255

# DeepSHAP Implementation

In [ ]:
"""
Begin
"""

In [ ]:
working_dir = '/content/drive/MyDrive/EECS 545 project/'
mnist_model = keras.models.load_model(working_dir + 'dense_layers_mnist_model_2')

In [ ]:
w2_weights = mnist_model.layers[1].get_weights()[0]
w2_bias = mnist_model.layers[1].get_weights()[1]

w1_weights = mnist_model.layers[0].get_weights()[0]
w1_bias = mnist_model.layers[0].get_weights()[1]

In [ ]:
def sigmoid(x, derivative=False):
    if derivative:
        return (np.exp(-x))/((np.exp(-x)+1)**2)
    return 1/(1 + np.exp(-x))

def relu(x, derivative=False):
    if derivative:
      if x >= 0:
          return 1
      else:
          return 0
    else:
      return (x >= 0) * x

def softmax(x, derivative=False):
    # Numerically stable with large exponentials
    # shape: (samples, NUM_CLASSES)
    exps = np.exp(x - x.max(axis=1).reshape(-1, 1))
    if derivative:
        return np.multiply((exps / np.sum(exps, axis=1).reshape(-1, 1)), (1 - exps / np.sum(exps, axis=1).reshape(-1, 1)))
    
    return exps / np.sum(exps, axis=1).reshape(-1, 1)


# Original Image

In [ ]:
# index = 4568
# index = 7
index = 134
probs = list(mnist_model.predict(np.array([X_test_preprocessed[index]]))[0])
print(probs.index(max(probs)))
print(max(probs))

fig, axs = plt.subplots(1, 1)
axs.imshow(X_test_preprocessed[index].reshape(28,28))

# fig, axs = plt.subplots(1,8)
# idx = 0
# for i in range(0, 200):
#     if Y_test[i] == 8:
#       print(i)
#       axs[idx].imshow(X_test_preprocessed[i].reshape(28,28))
#       idx += 1

In [ ]:
input_img = X_test_preprocessed[index].reshape(1, -1)
l1_pre = np.matmul(input_img, w1_weights) + w1_bias.reshape(1, -1)
l1 = relu(l1_pre, derivative=False)
l2_pre = np.matmul(l1, w2_weights) + w2_bias.reshape(1, -1)
l2 = softmax(l2_pre, derivative=False)
max_idx = np.argmax(l2)
max_probability = l2.max()
print("Max probability number: ", max_idx)
print( "Max probability: ", max_probability)

# Reference Image

In [ ]:
background_data = X_test_preprocessed[0:1000]
expected_X = np.mean(background_data, axis=0).reshape(1, -1)
fig, axs = plt.subplots(1,1)
axs.imshow(expected_X.reshape(28,28))

In [ ]:
# l1_blank_pre_data = np.matmul(background_data, w1_weights) + w1_bias.reshape(1, -1)
# l1_blank_pre = l1_blank_pre_data.copy().mean(axis=0).reshape(1, -1)
# # l1_blank_pre = np.matmul(input_img, w1_weights) + w1_bias.reshape(1, -1)

# l1_blank_data = relu(l1_blank_pre_data, derivative=False)
# l1_blank = l1_blank_data.copy().mean(axis=0).reshape(1, -1)
# # l1_blank = relu(l1_blank_pre, derivative=False)

# l2_blank_pre_data = np.matmul(l1_blank_data, w2_weights) + w2_bias.reshape(1, -1)
# l2_blank_pre = l2_blank_pre_data.copy().mean(axis=0).reshape(1, -1)
# # l2_blank_pre = np.matmul(l1_blank, w2_weights) + w2_bias.reshape(1, -1)

# l2_blank_data = softmax(l2_blank_pre_data, derivative=False)
# l2_blank = l2_blank_data.copy().mean(axis=0).reshape(1, -1)
# # l2_blank = softmax(l2_blank_pre, derivative=False)

l1_blank_pre = np.matmul(expected_X, w1_weights) + w1_bias.reshape(1, -1)
l1_blank = relu(l1_blank_pre, derivative=False)
l2_blank_pre = np.matmul(l1_blank, w2_weights) + w2_bias.reshape(1, -1)
l2_blank = softmax(l2_blank_pre.reshape(1, -1), derivative=False)

# Delta

In [ ]:
print(l2_blank)
print(l2_blank[0, max_idx])

l2_delta = l2-l2_blank
l2_pre_delta = l2_pre - l2_blank_pre
l1_delta = l1 - l1_blank
l1_pre_delta = l1_pre - l1_blank_pre
x_delta = input_img - expected_X

print(l2_delta)

# Multipliers

In [ ]:
multiplier_l1_l2_pre = w2_weights
print(multiplier_l1_l2_pre.shape)

In [ ]:
multiplier_l1_pre_l1 = np.zeros((1, l1_pre_delta.shape[1]))
if np.sum(abs(l1_pre_delta.squeeze())<1e-6) > 0:
    multiplier_l1_pre_l1[:, l1_pre_delta.squeeze()==0] = relu(l1_pre[:, l1_pre_delta.squeeze()==0], derivative=True)
if np.sum(abs(l1_pre_delta.squeeze())>1e-6) > 0:
    multiplier_l1_pre_l1[:, l1_pre_delta.squeeze()!=0] = l1_delta[:, l1_pre_delta.squeeze()!=0] / l1_pre_delta[:, l1_pre_delta.squeeze()!=0]

In [ ]:
multiplier_l2_pre_l2 = np.zeros((1, l2_pre_delta.shape[1]))
if np.sum(abs(l2_pre_delta.squeeze())<1e-6) > 0:
    multiplier_l2_pre_l2[:, l2_pre_delta.squeeze()==0] = softmax(l2_pre[:, l2_pre_delta.squeeze()==0], derivative=True)
if np.sum(abs(l2_pre_delta.squeeze())>1e-6) > 0:
    multiplier_l2_pre_l2[:, l2_pre_delta.squeeze()!=0] = l2_delta[:, l2_pre_delta.squeeze()!=0] / l2_pre_delta[:, l2_pre_delta.squeeze()!=0]

In [ ]:
#  chain rule
multiplier_l1_pre_l2_pre = multiplier_l1_l2_pre * multiplier_l1_pre_l1.T  # shape (32, 10)
multiplier_x_l2_pre = np.matmul(w1_weights, multiplier_l1_pre_l2_pre)  # shape (784,10)
multiplier_x_l2 = multiplier_x_l2_pre * multiplier_l2_pre_l2

In [ ]:
explaination = multiplier_x_l2_pre * x_delta.T  # shape (784,10)
print(explaination.sum(axis=0))
print(l2_pre_delta)

In [ ]:
#  Normalization
explaination = explaination - explaination.mean(axis=1).reshape(-1, 1)

In [ ]:
number_compare_list = [3, 4, 5, 6, 7]
fig, axs = plt.subplots(3, len(number_compare_list))
number = max_idx
limit_common = abs(explaination).max()
axs[0, 0].imshow(input_img.reshape(28,28), cmap='RdBu', vmin = -1, vmax=1)
axs[0, 1].imshow(expected_X.reshape(28,28), cmap='RdBu', vmin = -1, vmax=1)
axs[0, 2].imshow(explaination[:, number].reshape(28,28), cmap='RdBu', vmin = -limit_common, vmax=limit_common)
axs[0, 0].axis("off")
axs[0, 1].axis("off")
axs[0, 2].axis("off")
axs[0, 3].axis("off")
axs[0, 4].axis("off")

for i, number_compare in enumerate(number_compare_list):
    # mean = explaination.mean()
    # axs[0].imshow(explaination[:, number].reshape(28,28), cmap='gray', vmin = explaination[:, number].min(), vmax=explaination[:, number].max())
    # axs[i, 0].imshow(input_img.reshape(28,28), cmap='RdBu', vmin = -1, vmax=1)
    # axs[i, 1].imshow(expected_X.reshape(28,28), cmap='RdBu', vmin = -1, vmax=1)
    # limit_common = max(abs(explaination[:, number]).max(), abs(explaination[:, number_compare]).max())
    # limit = abs(explaination[:, number]).max()
    # axs[i, 2].imshow(explaination[:, number].reshape(28,28), cmap='RdBu', vmin = -limit_common, vmax=limit_common)
    # limit = abs(explaination[:, number_compare]).max()
    axs[1, i].imshow(explaination[:, number_compare].reshape(28,28), cmap='RdBu', vmin = -limit_common, vmax=limit_common)
    axs[1, i].axis("off")
    sort_idx = np.argsort(explaination[:, number_compare])
    mask = sort_idx[0:round(explaination.shape[0] * 0.2)]
    # mask = (explaination[:, number_compare] < 0)
    input_img_masked = input_img.copy()
    input_img_masked[0, mask] = 0
    axs[2, i].imshow(input_img_masked.reshape(28,28), cmap='RdBu', vmin = -1, vmax=1)
    axs[2, i].text(12, 35, number_compare)
    axs[2, i].axis("off")

plt.savefig(working_dir + "mnist-nn-relu.pdf")

In [ ]:
print(abs(explaination[:, number]).max())

In [ ]:
"""
End
"""